Introduction 

- 76 attributes though only 14 are used (chest pain type, cholestoral, painexer, etc) 
- predicting the presence of heart disease in the patient on a scale from 0 to 4
- a good technique would be to simply predict any sort of presence (1,2,3) from absence (0)
- heart disease prediction classification model 
- our goal is to train a model to predict whether individuals have heart disease based on various clinical attributes using of the KNN classification algorithm 

- heart disease is a major cause of death in the world population - it causes one person to die every 34 seconds in the US, coming 229 billion dollars in medical care each year in the US 
- machine learning takes a set of data as inputs and learns to predict output values from it 
- the effects of a potential machine learning algorithm that could be used to predict the presence of heart disease based on risk factors like blood pressure, cholesterol, etc could revolutionize health care for heart disease patients and lead to early treatment before the disease progresses to fatal levels 

Preliminary Exploratory Data Analysis 



In [10]:
library(tidyverse)

heart_data <- read_delim("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/hungarian.data", delim = ",", col_names = FALSE)
heart_data

Rows: 2940 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): X1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1
<chr>
1254 0 40 1 1 0 0
-9 2 140 0 289 -9 -9 -9
0 -9 -9 0 12 16 84 0
0 0 0 0 150 18 -9 7
172 86 200 110 140 86 0 0
0 -9 26 20 -9 -9 -9 -9
-9 -9 -9 -9 -9 -9 -9 12
20 84 0 -9 -9 -9 -9 -9
-9 -9 -9 -9 -9 1 1 1


Methods 

- will conduct the data analysis by tidying the data, scaling the predictor variable data and then creating a recipe that can be used within the model 
- classifier will be based on chest pain, blood pressure, resting electrocardiographic results, num (need to state reasons why we chose those variables) 

Expected Outcomes and Significance 

- we expect to find that the classifier will predict individuals with variables that have ________ (based on our research on heart disease, what scale for each of the variables would lead to a prediction of positive for heart disease) 
- 
- future questions could be asking whether two diseases are correlated and using a model to predict that???
- significance by offering easier, faster and more accurate diagnosis without the need for diagnosis by a clinical practitioner, can prevent life threats and allow for early treatment among patients that have risk factors 